In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!gdown --id 1-nhHWlpKY-sMwCG8eRHOJquerzmycH6m

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:138: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1-nhHWlpKY-sMwCG8eRHOJquerzmycH6m
From (redirected): https://drive.google.com/uc?id=1-nhHWlpKY-sMwCG8eRHOJquerzmycH6m&confirm=t&uuid=6656339e-024d-4f50-b3de-cf19234ebeb3
To: /content/FramesPS1.zip
100% 1.65G/1.65G [00:16<00:00, 99.0MB/s]


In [ ]:
!cp FramesPS1.zip /content/drive/MyDrive

In [ ]:

!unzip /content/drive/MyDrive/FramesPS1.zip -d /content/


Archive:  /content/drive/MyDrive/FramesPS1.zip
   creating: /content/content/FramesPS1/frames-PS1/
  inflating: /content/content/FramesPS1/frames-PS1/frame_2688.jpg  
  inflating: /content/content/FramesPS1/frames-PS1/frame_2984.jpg  
  inflating: /content/content/FramesPS1/frames-PS1/frame_24.jpg  
  inflating: /content/content/FramesPS1/frames-PS1/frame_1756.jpg  
  inflating: /content/content/FramesPS1/frames-PS1/frame_284.jpg  
  inflating: /content/content/FramesPS1/frames-PS1/frame_672.jpg  
  inflating: /content/content/FramesPS1/frames-PS1/frame_204.jpg  
  inflating: /content/content/FramesPS1/frames-PS1/frame_21352.jpg  
  inflating: /content/content/FramesPS1/frames-PS1/frame_2712.jpg  
  inflating: /content/content/FramesPS1/frames-PS1/frame_2684.jpg  
  inflating: /content/content/FramesPS1/frames-PS1/frame_1564.jpg  
  inflating: /content/content/FramesPS1/frames-PS1/frame_12004.jpg  
  inflating: /content/content/FramesPS1/frames-PS1/frame_2112.jpg  
  inflating: /conten

In [ ]:
!cp -r content /content/drive/MyDrive

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

# Path to the directory containing the images
image_dir = "/content/drive/MyDrive/content/FramesPS1/frames-PS1"

# Path to the annotations CSV file
annotations_file = "/content/drive/MyDrive/bounding_boxes.csv"

In [ ]:
# Read annotations CSV file
annotations_df = pd.read_csv(annotations_file)

# Split data into train and validation sets
train_df, val_df = train_test_split(annotations_df, test_size=0.2, random_state=42)

# Create train and validation YAML files
train_yaml = {'train': []}
val_yaml = {'val': []}

for index, row in train_df.iterrows():
    image_path = os.path.join(image_dir, row['Image Name'])
    train_yaml['train'].append({'img_path': image_path, 'annotations': [{'class_id': 0, 'bbox': row[['xmin', 'ymin', 'xmax', 'ymax']].tolist()}]})

for index, row in val_df.iterrows():
    image_path = os.path.join(image_dir, row['Image Name'])
    val_yaml['val'].append({'img_path': image_path, 'annotations': [{'class_id': 0, 'bbox': row[['xmin', 'ymin', 'xmax', 'ymax']].tolist()}]})

# Save YAML files
with open('train.yaml', 'w') as f:
    f.write('\n'.join([str(item) for item in train_yaml.items()]))

with open('val.yaml', 'w') as f:
    f.write('\n'.join([str(item) for item in val_yaml.items()]))

# Download YOLOv5 pre-trained checkpoints
os.system("wget https://github.com/ultralytics/yolov5/releases/download/v5.0/yolov5s.pt")

# Train YOLOv5 using the pre-trained checkpoint
os.system("python train.py --img 640 --batch 16 --epochs 50 --data train.yaml --weights yolov5s.pt --project=your_project_name")

512

In [ ]:
import torch
from pathlib import Path
import cv2
import numpy as np

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Set device (GPU or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Set model to evaluation mode
model = model.to(device).eval()

# Sample image paths
image_paths = ['/content/drive/MyDrive/content/FramesPS1/frames-PS1/frame_0.jpg', '/content/drive/MyDrive/content/FramesPS1/frames-PS1/frame_100.jpg']  # Replace with actual image paths

# Process each image
for image_path in image_paths:
    # Load image
    img = cv2.imread(image_path)[:, :, ::-1]  # OpenCV uses BGR channels, so convert to RGB

    # Make prediction
    results = model(img)

    # Visualize predictions
    results.show()


In [ ]:
import torch
from pathlib import Path
import cv2
import numpy as np

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Set device (GPU or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Set model to evaluation mode
model = model.to(device).eval()

# Function to process video and save annotated video
def process_video(video_path, output_path):
    # Open video capture
    cap = cv2.VideoCapture(video_path)

    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Define codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    # Process each frame of the video
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert frame to RGB
        img = frame[:, :, ::-1]

        # Make prediction
        results = model(img)

        # Annotate frame with bounding boxes
        annotated_frame = results.render()[0]

        # Convert frame back to BGR
        annotated_frame = annotated_frame[:, :, ::-1]

        # Write annotated frame to output video
        out.write(annotated_frame)

        # Display annotated frame (optional)
        # cv2.imshow('Annotated Frame', annotated_frame)
        # if cv2.waitKey(1) & 0xFF == ord('q'):
        #     break

    # Release video capture and writer
    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Path to input video
video_path = '/content/drive/MyDrive/sample vid - Made with Clipchamp.mp4'

# Path to save annotated video
output_path = '/content/drive/MyDrive/output_video_annotated.mp4'

# Process video and save annotated video
process_video(video_path, output_path)


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-3-16 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 224 layers, 7266973 parameters, 0 gradients
Adding AutoShape... 


In [ ]:
import torch
import cv2

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Set device (GPU or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Set model to evaluation mode
model = model.to(device).eval()

# Path to the input video
video_path = '/content/drive/MyDrive/sample vid - Made with Clipchamp.mp4'  # Replace with the path to your video file

# Define output video settings
output_path = '/content/drive/MyDrive/output_video.mp4'  # Replace with the path where you want to save the output video
video = cv2.VideoCapture(video_path)
frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(video.get(cv2.CAP_PROP_FPS))
codec = cv2.VideoWriter_fourcc(*'mp4v')
output_video = cv2.VideoWriter(output_path, codec, fps, (frame_width, frame_height))

# Function to draw lanes on the frame
def draw_lanes(image):
    # Lane 1
    cv2.line(image, (1141, 1353), (323, 1266), (0, 255, 0), 5)
    # Lane 2
    cv2.line(image, (1642, 1387), (1202, 1398), (0, 255, 0), 5)
    # Lane 3
    cv2.line(image, (2016, 1031), (1668, 1055), (0, 255, 0), 5)
    # Lane 4
    cv2.line(image, (2429, 1008), (2036, 1055), (0, 255, 0), 5)

# Process each frame in the video
frame_count = 0
while True:
    ret, frame = video.read()
    if not ret:
        print("End of video.")
        break

    frame_count += 1
    print(f"Processing frame {frame_count}...")

    try:
        # Perform object detection on the frame
        results = model(frame)

        # Draw lanes on the frame
        draw_lanes(frame)

        # Get the rendered image with bounding boxes
        rendered_image = results.render()[0]

        # Combine the rendered image with the original frame
        combined_frame = cv2.addWeighted(frame, 0.5, rendered_image, 0.5, 0)

        # Write the combined frame to the output video
        output_video.write(combined_frame)

        print(f"Frame {frame_count} written to output video.")

    except Exception as e:
        print(f"Error processing frame {frame_count}: {e}")

# Release video capture and writer
video.release()
output_video.release()
cv2.destroyAllWindows()


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-3-16 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 224 layers, 7266973 parameters, 0 gradients
Adding AutoShape... 


Processing frame 1...
Frame 1 written to output video.
Processing frame 2...
Frame 2 written to output video.
Processing frame 3...
Frame 3 written to output video.
Processing frame 4...
Frame 4 written to output video.
Processing frame 5...
Frame 5 written to output video.
Processing frame 6...
Frame 6 written to output video.
Processing frame 7...
Frame 7 written to output video.
Processing frame 8...
Frame 8 written to output video.
Processing frame 9...
Frame 9 written to output video.
Processing frame 10...
Frame 10 written to output video.
Processing frame 11...
Frame 11 written to output video.
Processing frame 12...
Frame 12 written to output video.
Processing frame 13...
Frame 13 written to output video.
Processing frame 14...
Frame 14 written to output video.
Processing frame 15...
Frame 15 written to output video.
Processing frame 16...
Frame 16 written to output video.
Processing frame 17...
Frame 17 written to output video.
Processing frame 18...
Frame 18 written to output

In [ ]:
import cv2
import numpy as np
import torch

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Function to check if a point is below the line
def below_line(point, line_start, line_end):
    x1, y1 = line_start
    x2, y2 = line_end
    x, y = point
    return (x - x1) * (y2 - y1) - (y - y1) * (x2 - x1) > 0

# Process each frame in the video
video_path = '/path/to/your/video.mp4'
video = cv2.VideoCapture(video_path)
while True:
    ret, frame = video.read()
    if not ret:
        break  # End of video

    # Get frame height and width
    frame_height, frame_width = frame.shape[:2]

    # Draw lines dividing the bottom into four equal parts
    line1_start = (0, frame_height * 3 // 4)
    line1_end = (frame_width, frame_height * 3 // 4)
    line2_start = (0, frame_height // 2)
    line2_end = (frame_width, frame_height // 2)
    line3_start = (0, frame_height // 4)
    line3_end = (frame_width, frame_height // 4)

    # Draw lines on the frame
    cv2.line(frame, line1_start, line1_end, (0, 255, 0), thickness=2)
    cv2.line(frame, line2_start, line2_end, (0, 255, 0), thickness=2)
    cv2.line(frame, line3_start, line3_end, (0, 255, 0), thickness=2)

    # Perform object detection on the frame
    results = model(frame)

    # Draw bounding boxes around detected vehicles
    for box in results.xyxy[0].cpu().numpy():
        x1, y1, x2, y2, _ = box
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)

    # Display the frame
    cv2.imshow('Frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture
video.release()
cv2.destroyAllWindows()


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-3-16 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 224 layers, 7266973 parameters, 0 gradients
Adding AutoShape... 


In [ ]:
import cv2
import numpy as np
import torch
from google.colab.patches import cv2_imshow  # Import cv2_imshow for displaying images in Colab

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Define line coordinates for each line (right to left)
lines = [
    [(496, 594), (150, 555)],
    [(722, 584), (545, 578)],
    [(911, 535), (743, 527)],
    [(1176, 558), (953, 580)]
]

# Initialize vehicle counts for each line
vehicle_counts = [0] * len(lines)

# Function to check if a point is to the right of the line
def right_of_line(point, line_start, line_end):
    x1, y1 = line_start
    x2, y2 = line_end
    x, y = point
    return (x - x1) * (y2 - y1) - (y - y1) * (x2 - x1) > 0

# Process each frame in the video
video_path = '/content/drive/MyDrive/sample vid - Made with Clipchamp.mp4'
video = cv2.VideoCapture(video_path)
while True:
    ret, frame = video.read()
    if not ret:
        break  # End of video

    # Perform object detection on the frame
    results = model(frame)

    # Get bounding boxes of detected objects
    bboxes = results.xyxy[0].cpu().numpy()

    # Draw lines on the frame
    for line_idx, (line_start, line_end) in enumerate(lines):
        cv2.line(frame, line_start, line_end, (0, 255, 0), thickness=2)

    # Count vehicles crossing each line
    for bbox in bboxes:
        for line_idx, (line_start, line_end) in enumerate(lines):
            x1, y1, x2, y2 = bbox[:4]
            bbox_center = ((x1 + x2) / 2, (y1 + y2) / 2)
            if right_of_line(bbox_center, line_start, line_end):
                vehicle_counts[line_idx] += 1
            # Draw bounding boxes
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)

    # Display vehicle counts for each line on the frame
    for line_idx, count in enumerate(vehicle_counts):
        cv2.putText(frame, f'Line {line_idx+1} Vehicles: {count}', (20, 50 + line_idx * 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the frame using cv2_imshow
    cv2_imshow(frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture
video.release()
cv2.destroyAllWindows()

print(f'Total Vehicles Counts: {vehicle_counts}')


In [ ]:
import cv2
import numpy as np
import torch

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Define line coordinates for each line (right to left)
lines = [
    [(496, 594), (150, 555)],
    [(722, 584), (545, 578)],
    [(911, 535), (743, 527)],
    [(1176, 558), (953, 580)]
]

# Initialize vehicle counts for each line
vehicle_counts = [0] * len(lines)

# Dictionary to track object trackers for each vehicle
object_trackers = {}

# Function to check if a point is to the right of the line
def right_of_line(point, line_start, line_end):
    x1, y1 = line_start
    x2, y2 = line_end
    x, y = point
    return (x - x1) * (y2 - y1) - (y - y1) * (x2 - x1) > 0

# Process each frame in the video
video_path = '/content/drive/MyDrive/sample vid - Made with Clipchamp.mp4'
video = cv2.VideoCapture(video_path)

# Get the video's frame width and height
frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create VideoWriter object
output_path = '/content/drive/MyDrive/output_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, 30.0, (frame_width, frame_height))

# Downsample parameters
scale_factor = 0.6 # Adjust as needed
new_width = int(frame_width * scale_factor)
new_height = int(frame_height * scale_factor)

# Process each frame in the video
while True:
    ret, frame = video.read()
    if not ret:
        break  # End of video

    # Resize frame
    frame_resized = cv2.resize(frame, (new_width, new_height))

    # Perform object detection on the frame
    results = model(frame_resized)

    # Get bounding boxes of detected objects
    bboxes = results.xyxy[0].cpu().numpy()

    # Iterate through detected vehicles
    for bbox in bboxes:
        x1, y1, x2, y2 = bbox[:4]
        bbox_center = ((x1 + x2) / 2, (y1 + y2) / 2)

        # Check if the bounding box is on the right side of any line
        for line_idx, (line_start, line_end) in enumerate(lines):
            if right_of_line(bbox_center, line_start, line_end):
                vehicle_id = f'{line_idx}_{int(bbox[0])}_{int(bbox[1])}_{int(bbox[2])}_{int(bbox[3])}'
                if vehicle_id not in object_trackers:
                    # Create an object tracker for the vehicle
                    tracker = cv2.TrackerKCF_create()
                    bbox_width = int(bbox[2] - bbox[0])
                    bbox_height = int(bbox[3] - bbox[1])
                    tracker.init(frame_resized, (int(bbox[0]), int(bbox[1]), bbox_width, bbox_height))
                    object_trackers[vehicle_id] = tracker

    # Update object trackers and count vehicles that cross the line
    for vehicle_id, tracker in object_trackers.items():
        success, bbox = tracker.update(frame_resized)
        if success:
            # Draw bounding box
            bbox = tuple(map(int, bbox))
            cv2.rectangle(frame_resized, (bbox[0], bbox[1]), (bbox[0] + bbox[2], bbox[1] + bbox[3]), (255, 0, 0), 2)

            # Check if the vehicle crosses the line
            x1, y1, width, height = bbox
            bbox_center = (x1 + width / 2, y1 + height / 2)
            for line_idx, (line_start, line_end) in enumerate(lines):
                if right_of_line(bbox_center, line_start, line_end):
                    if x1 < line_end[0] and x1 + width > line_start[0]:
                        vehicle_counts[line_idx] += 1
                        del object_trackers[vehicle_id]
                        break

    # Draw lines and display vehicle counts for each line on the frame
    for line_idx, (line_start, line_end) in enumerate(lines):
        cv2.line(frame_resized, line_start, line_end, (0, 255, 0), thickness=2)
        cv2.putText(frame_resized, f'Line {line_idx+1} Vehicles: {vehicle_counts[line_idx]}', (20, 50 + line_idx * 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Resize frame back to original size before writing to output video
    frame_resized_original_size = cv2.resize(frame_resized, (frame_width, frame_height))

    # Write the frame to the output video
    out.write(frame_resized_original_size)

# Release video capture and writer objects
video.release()
out.release()

print(f'Output video saved at: {output_path}')


/usr/local/lib/python3.10/dist-packages/torch/hub.py:294: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


requirements: Ultralytics requirement ['gitpython>=3.1.30'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 199.7 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 7.9s, installed 1 package: ['gitpython>=3.1.30']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



YOLOv5 🚀 2024-3-17 Python-3.10.12 torch-2.2.1+cu121 CPU

100%|██████████| 14.1M/14.1M [00:00<00:00, 89.3MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Output video saved at: /content/drive/MyDrive/output_video.mp4


In [ ]:
import cv2
import numpy as np
import torch
from google.colab.patches import cv2_imshow

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Define line coordinates for each line (right to left)
lines = [
    [(496, 594), (150, 555)],
    [(722, 584), (545, 578)],
    [(911, 535), (743, 527)],
    [(1176, 558), (953, 580)]
]

# Initialize vehicle counts for each line
vehicle_counts = [0] * len(lines)

# Dictionary to track vehicle positions
vehicle_positions = {}

# Minimum distance threshold to consider as separate vehicles
min_distance_threshold = 50

# Function to check if a point is to the right of the line
def right_of_line(point, line_start, line_end):
    x1, y1 = line_start
    x2, y2 = line_end
    x, y = point
    return (x - x1) * (y2 - y1) - (y - y1) * (x2 - x1) > 0

# Function to check if a bounding box intersects with a line
def bbox_intersects_line(bbox, line_start, line_end):
    x1, y1, x2, y2 = bbox
    bbox_center = ((x1 + x2) / 2, (y1 + y2) / 2)
    return right_of_line(bbox_center, line_start, line_end)

# Function to calculate distance between two points
def calculate_distance(point1, point2):
    x1, y1 = point1
    x2, y2 = point2
    return np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

# Process each frame in the video
video_path = '/content/drive/MyDrive/sample vid - Made with Clipchamp.mp4'
video = cv2.VideoCapture(video_path)

while True:
    ret, frame = video.read()
    if not ret:
        break  # End of video

    # Draw lines on the frame
    for line_start, line_end in lines:
        cv2.line(frame, line_start, line_end, (0, 255, 0), thickness=2)

    # Perform object detection on the frame
    results = model(frame)

    # Get bounding boxes of detected objects
    bboxes = results.xyxy[0].cpu().numpy()

    # Iterate through detected vehicles
    for bbox in bboxes:
        # Iterate through each line and check if the bounding box intersects with it
        for line_idx, (line_start, line_end) in enumerate(lines):
            if bbox_intersects_line(bbox[:4], line_start, line_end):
                vehicle_center = ((bbox[0] + bbox[2]) / 2, (bbox[1] + bbox[3]) / 2)

                # Check if the vehicle has been counted before
                vehicle_id = f'{line_idx}_{int(vehicle_center[0])}_{int(vehicle_center[1])}'
                if vehicle_id not in vehicle_positions:
                    vehicle_positions[vehicle_id] = vehicle_center
                    vehicle_counts[line_idx] += 1
                else:
                    # Check if the vehicle has moved significantly from the previously counted position
                    prev_vehicle_center = vehicle_positions[vehicle_id]
                    distance = calculate_distance(vehicle_center, prev_vehicle_center)
                    if distance > min_distance_threshold:
                        vehicle_positions[vehicle_id] = vehicle_center
                        vehicle_counts[line_idx] += 1

    # Display vehicle counts for each line on the frame
    for line_idx, (line_start, line_end) in enumerate(lines):
        cv2.putText(frame, f'Line {line_idx+1} Vehicles: {vehicle_counts[line_idx]}', (20, 50 + line_idx * 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the frame
    cv2_imshow(frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture
video.release()

print('Vehicle counts for each line:')
for line_idx, count in enumerate(vehicle_counts):
    print(f'Line {line_idx+1}: {count}')
